# OSeMOSYS-PLEXOS global model: Powerplant data

### Import modules

In [7]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import itertools

%reload_ext blackcellmagic

### Import data files and user input

In [8]:
## Data for variable costs of fuels taken from World Bank Commodity Market Outlooks:
##    https://www.worldbank.org/en/research/commodity-markets
## Download the 'Charts and Data' zip file and extract the forecasts file (CMO-April-2020-forecasts.xlsx).
## Adjust filename below for forecasts done at a different date.

# Read in World Bank Commodity Price Outlook - we only want rows 87 - 91
# using 85 as the headers (years) and skipping the energy header...

#### REPLACE FROM BELOW ONCE FIGURED OUT...df_prices = pd.read_excel(
df_prices = pd.read_excel(
    "data/CMO-April-2020-forecasts.xlsx", header=1, skiprows=83, nrows=6
)

# Read in Technologies
df_techs = pd.read_csv(r"osemosys_global_model/data/TECHNOLOGY.csv")

model_start_year = 2015
model_end_year = 2050
years = [range(model_start_year, model_end_year + 1)]

region_name = "GLOBAL"
emissions = []

### Filter technologies to keep only fuel production technologies (MIN)

In [9]:
df_techs = df_techs[df_techs.VALUE.str.contains('MIN')]

df_techs['REGION'] = region_name
df_techs['MODE_OF_OPERATION'] = 1
df_techs_temp = df_techs.copy()
df_techs_temp['MODE_OF_OPERATION'] = 2
df_techs = pd.concat([df_techs,df_techs_temp])
df_techs.rename(columns={'VALUE':'TECHNOLOGY'}, inplace=True)


df_techs['YEAR'] = [range(model_start_year, model_end_year+1)] * len(df_techs)

df_techs = df_techs.explode('YEAR')

df_techs.reset_index(drop=True, inplace=True)

# print(df_techs)

### Cleanup prices from CMO data

In [10]:
df_prices = df_prices.drop([0], axis=0)
df_prices = df_prices.drop(["Commodity"], axis=1)
columns = df_prices.columns.values
columns[0] = "YEAR"
df_prices = df_prices.loc[:, ~df_prices.columns.str.contains('^Unnamed', na=False)]

df_prices = df_prices.transpose()

# use commodity key as header
new_header = df_prices.iloc[0] #grab the first row for the header
df_prices = df_prices[1:] #take the data less the header row
new_header[1] = 'MINCOA'  # This was KFCOAL_AUS  KFCRUDE_PETRO  KFNGAS_EUR
new_header[2] = 'MINOIL'  # This was KFCRUDE_PETRO
new_header[3] = 'MINGAS'  # This was KFNGAS_EUR
df_prices.columns = new_header #set the header row as the df header

# drop units as we're doing $/PJ
df_prices = df_prices.drop("Unit", axis = 0)

# And convert to $/PJ
# Values taken from kylesconverter.com:
#  1 MT coal contains 29.31 PJ (1,000,000 tonnes coal) Original was $/mt ($/tonne I think).  We want $mill/PJ so divide by 29.31
#  1 MMBtu = 0.000001055056 PJ Original was MMBtu and we want $mill/pJ, so divide by 1.055056.
#  1 bbl = 0.00000612 PJ (Barrels of Oil) Original was $/bbl so divide by 0.00000612 and multiply by 1000000
# NOT ENTIRELY SURE I'M CALCULATING THE UNITS CORRECTLY (NOT SURE WHAT ORIGINAL UNITS WERE AS IT'S NOT STATED ANYWHERE)

# ORIGINAL UNITS:
#       KFCOAL_AUS KFCRUDE_PETRO KFNGAS_EUR KFNGAS_US KFNGAS_JP
#Unit       $/mt         $/bbl    $/mmbtu   $/mmbtu   $/mmbtu

df_prices['MINCOA'] = df_prices['MINCOA'] / 29.31
df_prices['MINOIL'] = df_prices['MINOIL'] / 0.00000612 / 1000000
df_prices['MINGAS'] = df_prices['MINGAS'] / 1.055056
df_prices['KFNGAS_US'] = df_prices['KFNGAS_US'] / 1.055056
df_prices['KFNGAS_JP'] = df_prices['KFNGAS_JP'] / 1.055056

print(df_prices)

# Add in other fuels that are the same as those above:
df_prices['MINCOG'] = df_prices['MINGAS']  # Cogen is powered by gas
df_prices['MINOTH'] = df_prices['MINOIL']  # Other petroleum products are similar to oil
df_prices['MINPET'] = df_prices['MINOIL']  # Petroleum products are similar to oil

# And add in international prices that are 15% higher than the regular ones:
df_prices['INTCOA'] = df_prices['MINCOA'] * 0.15
df_prices['INTOIL'] = df_prices['MINOIL'] * 0.15
df_prices['INTGAS'] = df_prices['MINGAS'] * 0.15
df_prices['INTCOG'] = df_prices['MINCOG'] * 0.15
df_prices['INTOTH'] = df_prices['MINOTH'] * 0.15
df_prices['INTPET'] = df_prices['MINPET'] * 0.15

df_prices = df_prices.reindex(range(model_start_year, model_end_year+1))

df_prices = df_prices.apply(pd.to_numeric)

df_prices = df_prices.interpolate()

col = df_prices.columns
df_prices = pd.melt(df_prices.reset_index(), id_vars='index', value_vars=col)

df_prices = df_prices.rename(columns={'index': 'YEAR', 'YEAR': 'TEMPTECH'})

# print(df_prices)

YEAR   MINCOA   MINOIL   MINGAS KFNGAS_US KFNGAS_JP
2014  2.21101  14.5306   8.8037     3.827   14.0448
2015  2.05491  8.47446  6.60675   2.53154   10.5909
2016  2.39914  7.43927  4.59973   2.51204   7.43109
2017   3.1032  8.86575  5.56786   2.88238   8.38155
2018  3.58755  10.9726  7.15352   2.93816   9.93625
2019  2.67126  10.0866  4.57563   2.44475   10.0656
2020  2.24031  5.77733  2.96823   1.91498   8.33018
2021  2.30686  6.82379  3.86399   2.16761   8.38769
2022  2.23751  7.10327  4.03303   2.26708   8.20741
2023  2.16897  7.38989  4.20702   2.36974   8.02633
2024  2.10155  7.68445  4.38643   2.47587   7.84554
2025   2.0354  7.98755  4.57168   2.58573   7.66576
2030  1.72663  9.64738   5.5961   3.19777   6.79526


### Map costs to technologies by region/country

In [11]:
## Need to create: REGION,TECHNOLOGY,MODE_OF_OPERATION,YEAR,VALUE

# Setup TEMPTECH column for merge
df_techs['TEMPTECH'] = df_techs['TECHNOLOGY'].str[0:6]

# International costs are identified by the INT in MINCOAINT, so make these INTCOA temporarily
df_techs.loc[df_techs.TECHNOLOGY.str[6:9]=='INT', 'TEMPTECH'] = 'INT'+df_techs['TECHNOLOGY'].str[3:6]

df_varcost = pd.merge(df_techs, df_prices, on=['YEAR', 'TEMPTECH'])

df_varcost = df_varcost.rename(columns={'value': 'VALUE'})

df_varcost = df_varcost.drop(['TEMPTECH'], axis=1)

# print(df_varcost)

## AND THEN DROP ANY ITEMS THAT HAVE A NAN AS THESE ARE DEFAULT (0) VALUES
## BUT THIS IS ALREADY DONE FOR US BY THE MERGE
#df_techs.dropna(subset = ["VALUE"], inplace=True)
#print(df_techs)

### Write out variablecost.csv

In [12]:
df_varcosts_final = df_varcost[['REGION', 
                       'TECHNOLOGY', 
                       'MODE_OF_OPERATION',
                       'YEAR', 
                       'VALUE']]

df_varcosts_final.to_csv(r"osemosys_global_model/data/VariableCost.csv", mode='a', header=False, index = None)

print('finished')

finished
